In [ ]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
import networkx as nx
import random
from networkx.algorithms import distance_regular
import random

In [4]:
random.seed(42)
torch.manual_seed(42)

In [5]:
dataset = TUDataset(root='/tmp/AIDS', name='AIDS', use_node_attr=True)
print(f"Loaded {len(dataset)} graphs.")

Loaded 2000 graphs.


In [11]:
def weisfeiler_lehman_hash(graph):
    colors = {node: data.get('label', 0) for node, data in graph.nodes(data=True)}
    for _ in range(len(graph.nodes())):
        new_colors = {}
        for node in graph.nodes():
            neighbor_colors = sorted([colors[nbr] for nbr in graph.neighbors(node)])
            signature = (colors[node], tuple(neighbor_colors))
            new_colors[node] = hash(signature)
        if new_colors == colors:
            break
        colors = new_colors
    return str(sorted(colors.values()))


In [6]:
network_x_graphs=[]
for i,data in enumerate(dataset):
    g= to_networkx(data,node_attrs=['x'])
    g= g.to_undirected()
    nx.set_node_attributes(g,{j:int(x[0]) for j,x in enumerate(data.x)},'label')
    network_x_graphs.append(g)

In [7]:
srg_list=[]#list to store strongly regular graphs
for index, g in enumerate(network_x_graphs):
    try:
        result= distance_regular.is_strongly_regular(g)
        if result is not False:
            srg_list.append((index,result))
    except Exception as e:
        pass

In [8]:
print(f"Total graphs checked: {len(network_x_graphs)}")
print("Strongly regular graphs found:", srg_list)

Total graphs checked: 2000
Strongly regular graphs found: [(1482, True)]


In [9]:
graph_index= 1482
data = dataset[graph_index]
G = to_networkx(data, to_undirected=True)
nx.set_node_attributes(G, {j:int(x[0]) for j,x in enumerate(data.x)},'label')
print(f"Graph {graph_index} has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

Graph 1482 has 5 nodes and 5 edges


In [12]:
original_hash = weisfeiler_lehman_hash(G)
edges= list(G.edges())
non_edges= list(nx.non_edges(G))
u,v = random.choice(edges)

In [13]:
G_removed= G.copy()
G_removed.remove_edge(u,v)
x,y =random.choice(non_edges)
G_added = G.copy()
G_added.add_edge(x,y)

In [14]:
GM_removed = nx.isomorphism.GraphMatcher(G,G_removed,node_match=lambda a,b: a['label']==b['label'])
GM_added = nx.isomorphism.GraphMatcher(G,G_added,node_match=lambda a,b: a['label']==b['label'])
print(f"Original vs Removed-edge graph: {GM_removed.is_isomorphic()}")
print(f"Original vs Added-edge graph  : {GM_added.is_isomorphic()}")

Original vs Removed-edge graph: False
Original vs Added-edge graph  : False
